# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# PSNR

In [ ]:
import math
import numpy as np

def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    # img1 = img1.astype(np.float64)
    # img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))

# SSIM

In [ ]:
import math
import numpy as np
import cv2

def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

# SSIM by CV2 lib

In [ ]:
from skimage.metrics import structural_similarity
import cv2
import numpy as np

def calculate_ssim_by_lib(img1, img2):

  # Convert images to grayscale
  before_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
  after_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

  # Compute SSIM between two images
  (score, diff) = structural_similarity(before_gray, after_gray, full=True)

  return score

# PSNR by CV2 lib

In [ ]:
def calculate_psnr_by_lib(img1, img2):
  return cv2.PSNR(img1, img2)

# Result

# Evaluate one image

In [ ]:
import cv2

# read an image
img1 = cv2.imread("/content/drive/MyDrive/Colab Notebooks/original/000.jpg")
img2 = cv2.imread("/content/drive/MyDrive/Colab Notebooks/output/000.png")

# Get the size of the second image
height, width, _ = img1.shape

# Resize the first image to the same size as the second image
img2 = cv2.resize(img2, (width, height))

psnr = calculate_psnr_by_lib(img1, img2)
print(f'psnr={psnr}')

ssim = calculate_ssim_by_lib(img1, img2)
print(f'ssim={ssim}')

# Evaluate images in the folder

In [ ]:
import cv2

n = 100
psnrSum = 0.0
ssimSum = 0.0
for i in range(n):
  fileName = str(i).zfill(3)
  # read an image
  img1 = cv2.imread(f"/content/drive/MyDrive/Colab Notebooks/original/{fileName}.jpg")
  img2 = cv2.imread(f"/content/drive/MyDrive/Colab Notebooks/output/{fileName}.png")

  # Get the size of the second image
  height, width, _ = img1.shape

  # Resize the first image to the same size as the second image
  img2 = cv2.resize(img2, (width, height))

  print(f'fileName:{fileName}')

  psnr = calculate_psnr_by_lib(img1, img2)
  print(f'psnr={psnr}')

  ssim = calculate_ssim_by_lib(img1, img2)
  print(f'ssim={ssim}')

  psnrSum += psnr
  ssimSum += ssim

print(f'Mean of PSNR={psnrSum/n}')
print(f'Mean of SSIM={ssimSum/n}')

# Add Noise

In [ ]:
import cv2
import numpy as np

# Load the image
img = cv2.imread(f"/content/drive/MyDrive/Colab Notebooks/resized/000.jpg")

# Add Gaussian noise to the image
mean = 0
variance = 100
sigma = np.sqrt(variance)
gaussian = np.random.normal(mean, sigma, img.shape)
noisy_img = np.zeros(img.shape, np.float32)
if len(img.shape) == 2:
    noisy_img = img + gaussian
else:
    noisy_img[:, :, 0] = img[:, :, 0] + gaussian[:, :, 0]
    noisy_img[:, :, 1] = img[:, :, 1] + gaussian[:, :, 1]
    noisy_img[:, :, 2] = img[:, :, 2] + gaussian[:, :, 2]

# Normalize pixel values to lie between 0 and 255
cv2.normalize(noisy_img, noisy_img, 0, 255, cv2.NORM_MINMAX, dtype=-1)

# Convert the pixel values to integers
noisy_img = noisy_img.astype(np.uint8)

# Save the noisy image
cv2.imwrite(f"/content/drive/MyDrive/Colab Notebooks/resized/noise_000.jpg", noisy_img)

True